In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('C:\\Users\\koolk\\Desktop\\brain-diffusion\\Chad_functions_and_unittests'))
if module_path not in sys.path:
    sys.path.append(module_path)

from trajectory_visualization import plot_trajectory, sidebyside, shift_trajectory, overlay, shift_trajectory3D
from trajectory_visualization import plot_trajectories3D, plot_3Doverlay, plot_MSDorDeff, plot_MeanMSDorDeff, randtraj, multrandtraj
from trajectory_visualization import randtraj2, plot_Mean2DMSDsorDeff, plot_MSDorDeffLR, LRfor3D2D, fillin, prettify
from methods_of_Dcalc import Dpointder, Dlinalg, Dlinalgw
from Diffusion_Functions import frame_adjust, MSD_Deff_Calcs, prettify2, fillin2

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import scipy.optimize as opt
import scipy.stats as stat
from mpl_toolkits.mplot3d import Axes3D
from operator import itemgetter
import random
import numpy as np
import numpy.ma as ma
import numpy.linalg as la
from mpl_toolkits.mplot3d import Axes3D

pi = np.pi
sin = np.sin
cos = np.cos

In [2]:
name = '100nm_RED_PEG_NPs_in_1_2p_agarose'

to_show = 20

geoM1x = np.genfromtxt('geoM1x_{}.csv'.format(name), delimiter =",")[0:to_show]
geoM1y = np.genfromtxt('geoM1y_{}.csv'.format(name), delimiter =",")[0:to_show]
geoM2xy = np.genfromtxt('geoM2xy_{}.csv'.format(name), delimiter =",")[0:to_show]

geoM1x[0] = 0
geoM1y[0] = 0
geoM2xy[0] = 0

In [3]:
conversion = dict()

conversion[1] = (0.10, 9.91, 1)

frames = 446
frames_1 = np.linspace(0, frames, frames+1).astype(np.int64)
time = frames_1/conversion[1][1]
time = time[1:to_show+1]
logplot = '{}_logplot'.format(name)
Mplot = '{}_Mplot'.format(name)
Dplot = '{}_Dplot'.format(name)
Hplot = '{}_Hplot'.format(name)
Hlogplot = '{}_Hlogplot'.format(name)
Cplot = '{}_Cplot'.format(name)
Tplot = '{}_Tplot'.format(name)
T2plot = '{}_T2plot'.format(name)

In [4]:
def plot_MLOG(M2xy, Mx, My, time, dec1, dec2, datatype, filename, limit1, limit2, tick1, tick2):
    """
    Plots the MSDs or Deffs from a trajectory dataset.

    n1: particle numbers (typically 0)
    n2: time (typically 1)
    n3: MSDs or Deffs (11 or 15 typically)
    """

    M2 = M2xy
    Mx = Mx
    My = My
    t = time
    
    #I will need to calculate standard deviations eventually
#     #Now to calculate the standard dev at each point:
#     for num in range (1, total1):
#         SDunit = (MSD[num] - MMSD)**2
#         SD = SD + SDunit
#     SD = np.sqrt(SD/total1)
#     SE = SD/np.sqrt(total1)
    
#     #Linear algebra to find Deff:
#     t = time[1][:]
#     w = dict()
#     line = dict()
#     A = np.ones((np.shape(t)[0], 2))
#     A[:, 0] = t
#     w[0] = np.linalg.lstsq(A, MMSD[:, 0])[0]
#     w[1] = np.linalg.lstsq(A, MMSD[:, 1])[0]
#     w[2] = np.linalg.lstsq(A, MMSD[:, 2])[0]
#     line[0] = w[0][0]*t + w[0][1]
#     line[1] = w[1][0]*t + w[1][1]
#     line[2] = w[2][0]*t + w[2][1]
    
#     #Linear algebra for fit on log plot:
#     wl = dict()
#     linel = dict()
#     lt = np.log(t)
#     lA = np.ones((np.shape(t)[0], 2))
#     lA[:, 0] = lt
#     lM = np.log(MMSD)
#     wl[0] = np.linalg.lstsq(lA, lM[:, 0])[0]
#     wl[1] = np.linalg.lstsq(lA, lM[:, 1])[0]
#     wl[2] = np.linalg.lstsq(lA, lM[:, 2])[0]
#     linel[0] = np.exp(wl[0][0]*lt + wl[0][1])
#     linel[1] = np.exp(wl[1][0]*lt + wl[1][1])
#     linel[2] = np.exp(wl[2][0]*lt + wl[2][1])

    # Creates figure
    fig = plt.figure(figsize=(24, 18), dpi=80)
    ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

    ax.plot(t[1:], M2[1:], linestyle='-', linewidth=7, label='2D', marker='o', ms=15, color='blue')
    ax.plot(t[1:], Mx[1:], linestyle='-', linewidth=7, label='1D x', marker='o', ms=15, color='red')
    ax.plot(t[1:], My[1:], linestyle='-', linewidth=7, label='1D y', marker='o', ms=15, color='green')
#     ax.plot(t[1:], Mth[1:], '--', linewidth=10, label='theoretical', color='purple')
    
#     ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
#     ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
#     ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')
    
    
#     ax.plot(t, linel[0], linewidth=3, color='blue')
#     ax.plot(t, linel[1], linewidth=3, color='red')
#     ax.plot(t, linel[2], linewidth=3, color='green')
    
    # A few adjustments to prettify the graph
    for item in ([ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(70)

    xmajor_ticks = np.arange(0, limit1, tick1)
    ymajor_ticks = np.arange(0, limit2, tick2)

    hfont = {'fontname':'Arial'}
    ax.set_xticks(xmajor_ticks)
    ax.set_yticks(ymajor_ticks)
    ax.title.set_fontsize(70)
    ax.set_xlabel('Time (s)', fontsize=115, **hfont)
    ax.set_ylabel(r'MSD ($\mu$m$^2$)', fontsize=115, **hfont)
    ax.tick_params(direction='out', pad=16)
    ax.legend(loc=(0.05, 0.56), prop={'size': 70})
    plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))

    
    plt.yscale('log')
    plt.xscale('log')
    plt.gca().set_xlim([0.1, limit1])
    plt.gca().set_ylim([0.1, limit2])

    # Save your figure
    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return M2, Mx

In [10]:
"""
Plot of 2D, 1x, and 1y MSDs on a log scale.
"""

one1, bleh = plot_MLOG(geoM2xy, geoM1x, geoM1y, time, 1, 1, 'MSD (um^2)', logplot, 10.1, 10.1, 1, 1)
plt.show()

In [6]:
#Linear algebra to find Deff:

w = dict()
residuals = dict()
line = dict()
A1 = np.ones((np.shape(time[1:])[0], 2))
A1[:, 0] = time[1:]

w[0], residuals[0] = np.linalg.lstsq(A1[:, :], geoM2xy[1:])[0:2]
w[1], residuals[1] = np.linalg.lstsq(A1[:, :], geoM1x[1:])[0:2]
w[2], residuals[2] = np.linalg.lstsq(A1[:, :], geoM1y[1:])[0:2]


line[0] = w[0][0]*time + w[0][1]

In [7]:
print('2D Diffusion Coeff (um^2/s):', np.round(w[0][0]/4, 7), 'geoM2xy')
print('1Dx Diffusion Coeff (um^2/s):', np.round(w[1][0]/2, 7), 'geoM1x')
print('1Dy Diffusion Coeff (um^2/s):', np.round(w[2][0]/2, 7), 'geoM1y')


2D Diffusion Coeff (um^2/s): 0.2315797 geoM2xy
1Dx Diffusion Coeff (um^2/s): 0.1441642 geoM1x
1Dy Diffusion Coeff (um^2/s): 0.1626516 geoM1y


In [8]:
def plot_M(M2xy, Mx, My, time, dec1, dec2, datatype, filename, limit1, limit2, tick1, tick2):
    """
    Plots the MSDs or Deffs from a trajectory dataset.

    n1: particle numbers (typically 0)
    n2: time (typically 1)
    n3: MSDs or Deffs (11 or 15 typically)
    """

    M2 = M2xy
    Mx = Mx
    My = My
    t = time
    
#     #Linear algebra to find Deff:
#     t = time[1][:]
#     w = dict()
#     line = dict()
#     A = np.ones((np.shape(t)[0], 2))
#     A[:, 0] = t
#     w[0] = np.linalg.lstsq(A, MMSD[:, 0])[0]
#     w[1] = np.linalg.lstsq(A, MMSD[:, 1])[0]
#     w[2] = np.linalg.lstsq(A, MMSD[:, 2])[0]
#     line[0] = w[0][0]*t + w[0][1]
#     line[1] = w[1][0]*t + w[1][1]
#     line[2] = w[2][0]*t + w[2][1]
    
#     #Linear algebra for fit on log plot:
#     wl = dict()
#     linel = dict()
#     lt = np.log(t)
#     lA = np.ones((np.shape(t)[0], 2))
#     lA[:, 0] = lt
#     lM = np.log(MMSD)
#     wl[0] = np.linalg.lstsq(lA, lM[:, 0])[0]
#     wl[1] = np.linalg.lstsq(lA, lM[:, 1])[0]
#     wl[2] = np.linalg.lstsq(lA, lM[:, 2])[0]
#     linel[0] = np.exp(wl[0][0]*lt + wl[0][1])
#     linel[1] = np.exp(wl[1][0]*lt + wl[1][1])
#     linel[2] = np.exp(wl[2][0]*lt + wl[2][1])

    # Creates figure
    fig = plt.figure(figsize=(24, 18), dpi=80)
    ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

    ax.plot(t, M2, linestyle='-', linewidth=7, label='2D', marker='o', ms=15, color='blue')
    ax.plot(t, Mx, linestyle='-', linewidth=7, label='1D x', marker='o', ms=15, color='red')
    ax.plot(t, My, linestyle='-', linewidth=7, label='1D y', marker='o', ms=15, color='green')
#     ax.plot(t, Mth, '--', linewidth=10, label='theoretical', color='purple')
    
#     ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
#     ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
#     ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')
    
#     ax.plot(t, line[0], linewidth=3, color='blue')
#     ax.plot(t, line[1], linewidth=3, color='red')
#     ax.plot(t, line[2], linewidth=3, color='green')
    
    #ax.plot(t, linel[0], linewidth=3, color='blue')
    #ax.plot(t, linel[1], linewidth=3, color='red')
    #ax.plot(t, linel[2], linewidth=3, color='green')
    
    # A few adjustments to prettify the graph
    for item in ([ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(70)

    xmajor_ticks = np.arange(0, limit1, tick1)
    ymajor_ticks = np.arange(0, limit2, tick2)

    hfont = {'fontname':'Arial'}
    ax.set_xticks(xmajor_ticks)
    ax.set_yticks(ymajor_ticks)
    ax.title.set_fontsize(70)
    ax.set_xlabel('Time (s)', fontsize=115, **hfont)
    ax.set_ylabel(r'MSD ($\mu$m$^2$)', fontsize=115, **hfont)
    ax.tick_params(direction='out', pad=16)
    ax.legend(loc=(0.05, 0.56), prop={'size': 70})
    plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))

    
    #plt.yscale('log')
    #plt.xscale('log')
    plt.gca().set_xlim([0, limit1])
    plt.gca().set_ylim([0, limit2])

    # Save your figure
    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return M2, Mx

In [11]:
"""
Plots the 2D, 1x, and 1y MSDs on a normal scale.
"""

one1, bleh = plot_M(geoM2xy, geoM1x, geoM1y, time, 0, 1, 'MSD (um^2)', Mplot, 2.1, 8.1, 0.5, 2)
plt.show()